In [ ]:
import pandas as pd
df=pd.read_csv("C:/Users/MINA/Downloads/diabetes.csv")
df

# حساب توزيع الفئات
class_counts = df["diabetes"].value_counts()
print("قبل المعالجة:/n", class_counts)

# فصل الفئات
df_majority = df[df["diabetes"] == 0]  # الفئة الكبرى (91482)
df_minority = df[df["diabetes"] == 1]  # الفئة الصغرى (8500)

# اختيار عينة عشوائية من الفئة الكبرى بحجم الفئة الصغرى (8500)
df_majority_downsampled = df_majority.sample(n=len(df_minority), random_state=42)

# دمج البيانات مرة أخرى
df_balanced = pd.concat([df_majority_downsampled, df_minority])

# إعادة ترتيب البيانات عشوائيًا
df_balanced = df_balanced.sample(frac=1, random_state=42).reset_index(drop=True)

# التحقق من التوزيع بعد المعالجة
print("بعد المعالجة:/n", df_balanced["diabetes"].value_counts())


قبل المعالجة:
 0    91482
1     8500
Name: diabetes, dtype: int64
بعد المعالجة:
 1    8500
0    8500
Name: diabetes, dtype: int64


In [1]:
import pandas as pd
df=pd.read_csv("C:/Users/MINA/Downloads/diabetes.csv")
df


,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1
...,...,...,...,...,...,...,...,...,...
763,10,101,76,48,180,32.9,0.171,63,0
764,2,122,70,27,0,36.8,0.340,27,0
765,5,121,72,23,112,26.2,0.245,30,0
766,1,126,60,0,0,30.1,0.349,47,1


In [3]:
df["Outcome"].value_counts()

0    500
1    268
Name: Outcome, dtype: int64

In [2]:
df_balanced.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17000 entries, 0 to 16999
Data columns (total 9 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   gender               17000 non-null  object 
 1   age                  17000 non-null  float64
 2   hypertension         17000 non-null  int64  
 3   heart_disease        17000 non-null  int64  
 4   smoking_history      17000 non-null  object 
 5   bmi                  17000 non-null  float64
 6   HbA1c_level          17000 non-null  float64
 7   blood_glucose_level  17000 non-null  int64  
 8   diabetes             17000 non-null  int64  
dtypes: float64(3), int64(4), object(2)
memory usage: 1.2+ MB


In [3]:
df_balanced.duplicated().sum()

54

In [16]:
df_balanced.drop_duplicates(inplace=True)

In [5]:
df_balanced.duplicated().sum()

0

In [17]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report,confusion_matrix,ConfusionMatrixDisplay
from tensorflow.keras.optimizers import Adamax
import warnings
warnings.filterwarnings('ignore')

In [18]:
selected_feature=["HbA1c_level","blood_glucose_level","bmi","age","hypertension","heart_disease"]
X=df_balanced[selected_feature]
y=df_balanced["diabetes"]

In [19]:
#scaler=StandardScaler()
#X=scaler.fit_transform(X)

In [20]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:

param_grid = {
    'n_estimators': [50, 100, 200,300,500],  # عدد الأشجار
    'learning_rate': [0.01, 0.1, 0.2,0.9,0.6],  # معدل التعلم
    'max_depth': [3, 5, 7,9,12],  # عمق الشجرة
    'subsample': [0.8, 1],  # نسبة استخدام البيانات لكل شجرة
    'colsample_bytree': [0.8, 1]  # نسبة استخدام الميزات لكل شجرة
}

# إنشاء نموذج XGBClassifier
xgb_model = XGBClassifier()

# تطبيق GridSearchCV
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    scoring='accuracy',  # يمكنك تغييرها إلى f1 أو recall حسب الحاجة
    cv=5,  # عدد الـ folds
    n_jobs=-1,  # استخدام جميع المعالجات لتسريع العملية
    verbose=2
)

# **تمرير الأوزان يدويًا أثناء fit**
grid_search.fit(X_train, y_train)

# أفضل المعلمات
print("أفضل المعلمات:", grid_search.best_params_)

# أفضل نموذج بعد البحث
best_model = grid_search.best_estimator_

# التقييم
y_pred = best_model.predict(X_test)
print("\nAccuracy",accuracy_score(y_test, y_pred))

Fitting 5 folds for each of 256 candidates, totalling 1280 fits
أفضل المعلمات: {'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 5, 'n_estimators': 50, 'subsample': 1}

Accuracy 0.9117994100294985


In [10]:
model=best_model
y_predict=model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print("دقة النموذج:", accuracy)

print(classification_report(y_test, y_pred))

NameError: name 'best_model' is not defined

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC

rf_model = RandomForestClassifier(class_weight="balanced", random_state=42)
rf_model.fit(X_train, y_train)



RandomForestClassifier(class_weight='balanced', random_state=42)

In [16]:
y_prrrrrr=rf_model.predict(X_test)
accuracy = accuracy_score(y_test, y_prrrrrr)
print("دقة النموذج:", accuracy)

print(classification_report(y_test, y_prrrrrr))

دقة النموذج: 0.8982300884955752
              precision    recall  f1-score   support

           0       0.90      0.90      0.90      1693
           1       0.90      0.90      0.90      1697

    accuracy                           0.90      3390
   macro avg       0.90      0.90      0.90      3390
weighted avg       0.90      0.90      0.90      3390



# ANN

In [18]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout,BatchNormalization
from tensorflow.keras.optimizers import Adamax

In [26]:
model = Sequential([


     Dense(64, activation="relu", input_shape=(X_train.shape[1],)),
     Dense(128, activation="relu"),
     Dense(256, activation="relu"),
     Dense(512, activation="relu"),
     Dense(512, activation="relu"),
     Dropout(0.2),
     Dense(1024, activation="relu"),
     Dense(1024, activation="relu"),
     Dropout(0.2),
     BatchNormalization(),

     Dense(1, activation="sigmoid")  
])

model.compile(optimizer=Adamax(0.001), loss="binary_crossentropy", metrics=["accuracy"])

model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_13 (Dense)                │ (None, 64)             │           448 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_14 (Dense)                │ (None, 128)            │         8,320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_15 (Dense)                │ (None, 256)            │        33,024 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_16 (Dense)                │ (None, 512)            │       131,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_17 (Dense)                │ (None, 512)            │       262,656 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_4 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ (None, 1024)           │       525,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ (None, 1024)           │     1,049,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_5 (Dropout)             │ (None, 1024)           │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_2           │ (None, 1024)           │         4,096 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ (None, 1)              │         1,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,016,065 (7.69 MB)

 Trainable params: 2,014,017 (7.68 MB)

 Non-trainable params: 2,048 (8.00 KB)

In [ ]:
#model.fit(X_train, y_train, epochs=100, batch_size=8, validation_split=0.15, verbose=1)


Epoch 1/100
1441/1441 ━━━━━━━━━━━━━━━━━━━━ 26s 16ms/step - accuracy: 0.8401 - loss: 0.3602 - val_accuracy: 0.8953 - val_loss: 0.2140
Epoch 2/100
1441/1441 ━━━━━━━━━━━━━━━━━━━━ 24s 17ms/step - accuracy: 0.8634 - loss: 0.3008 - val_accuracy: 0.8958 - val_loss: 0.2319
Epoch 3/100
1441/1441 ━━━━━━━━━━━━━━━━━━━━ 22s 16ms/step - accuracy: 0.8813 - loss: 0.2611 - val_accuracy: 0.9031 - val_loss: 0.1947
Epoch 4/100
1441/1441 ━━━━━━━━━━━━━━━━━━━━ 22s 15ms/step - accuracy: 0.8837 - loss: 0.2644 - val_accuracy: 0.9046 - val_loss: 0.1939
Epoch 5/100
1441/1441 ━━━━━━━━━━━━━━━━━━━━ 22s 15ms/step - accuracy: 0.8901 - loss: 0.2512 - val_accuracy: 0.9027 - val_loss: 0.1947
Epoch 6/100
1441/1441 ━━━━━━━━━━━━━━━━━━━━ 23s 16ms/step - accuracy: 0.8903 - loss: 0.2553 - val_accuracy: 0.9095 - val_loss: 0.2000
Epoch 7/100
1441/1441 ━━━━━━━━━━━━━━━━━━━━ 23s 16ms/step - accuracy: 0.8889 - loss: 0.2457 - val_accuracy: 0.9051 - val_loss: 0.1949
Epoch 8/100
1441/1441 ━━━━━━━━━━━━━━━━━━━━ 24s 16ms/step - accuracy: 

KeyboardInterrupt: 

ValueError: None values not supported.

In [ ]:
#from flaml import AutoML

#automl = AutoML()
#automl.fit(X_train, y_train, task="classification", time_budget=600)  # التدريب لمدة 10 دقائق
#print("أفضل نموذج:", automl.best_estimator)

#from sklearn.metrics import classification_report
#y_pred = automl.predict(X_test)
#print(classification_report(y_test, y_pred))


[flaml.automl.logger: 03-15 16:37:58] {1728} INFO - task = classification
[flaml.automl.logger: 03-15 16:37:58] {1739} INFO - Evaluation method: cv
[flaml.automl.logger: 03-15 16:37:58] {1838} INFO - Minimizing error metric: 1-roc_auc
[flaml.automl.logger: 03-15 16:37:58] {1955} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth', 'sgd', 'catboost', 'lrl1']
[flaml.automl.logger: 03-15 16:37:58] {2258} INFO - iteration 0, current learner lgbm
[flaml.automl.logger: 03-15 16:37:59] {2393} INFO - Estimated sufficient time budget=1761s. Estimated necessary time budget=43s.
[flaml.automl.logger: 03-15 16:37:59] {2442} INFO -  at 0.3s,	estimator lgbm's best error=0.0528,	best estimator lgbm's best error=0.0528
[flaml.automl.logger: 03-15 16:37:59] {2258} INFO - iteration 1, current learner lgbm
[flaml.automl.logger: 03-15 16:37:59] {2442} INFO -  at 0.5s,	estimator lgbm's best error=0.0528,	best estimator lgbm's best error=0.0528
[flaml.automl.lo

In [21]:
from catboost import CatBoostClassifier


In [22]:
model = CatBoostClassifier(iterations=500, depth=6, learning_rate=0.1, verbose=False)
model.fit(X_train, y_train)


In [23]:
y_p=model.predict(X_test)
accuracy = accuracy_score(y_test, y_p)
print("دقة النموذج:", accuracy)

print(classification_report(y_test, y_p))

دقة النموذج: 0.8988200589970502
              precision    recall  f1-score   support

           0       0.91      0.88      0.90      1697
           1       0.89      0.91      0.90      1693

    accuracy                           0.90      3390
   macro avg       0.90      0.90      0.90      3390
weighted avg       0.90      0.90      0.90      3390



In [44]:
# إدخال بيانات شخص معين للاختبار
real_values = [[6, 120, 20, 45, 0, 0]]  # HbA1c_level, blood_glucose_level, bmi, age, hypertension, heart_disease


In [70]:
# توقع الاحتمالات للفئتين (الإصابة بالسكري أو عدمها)
probabilities = model.predict_proba(real_values)

# استخراج نسبة الإصابة بالسكري (العمود الثاني يمثل احتمالية الفئة 1 "Diabetic")
diabetes_probability = probabilities[0][1]

# طباعة النتيجة مع النسبة المئوية
print(f"التوقع: {'Diabetic' if diabetes_probability >= 0.5 else 'Non-Diabetic'} ({diabetes_probability * 100:.2f}%)")


التوقع: Non-Diabetic (0.01%)


In [48]:
import numpy as np
import catboost
from catboost import CatBoostClassifier

# Sample Data (6 features)
X_train = np.random.rand(1000, 6)  
y_train = np.random.randint(0, 2, size=(1000,))  

# Train CatBoost Model
catboost_model = CatBoostClassifier(iterations=100, depth=6, learning_rate=0.1, verbose=0)
catboost_model.fit(X_train, y_train)

# Get CatBoost Predictions (soft probabilities)
y_train_pred = catboost_model.predict_proba(X_train)  # Shape (1000, 2)


In [80]:
keras_model.save("C:/Users/MINA/Downloads/catboost_keras_model.h5")
print("Model saved as catboost_keras_modell.h5")


Model saved as catboost_keras_modell.h5


In [81]:
from tensorflow.keras.models import load_model
loaded_model = load_model("C:/Users/MINA/Downloads/catboost_keras_model.h5")

# Test with Real Input
real_values = np.array([[6, 120, 20, 45, 0, 0]])

In [82]:
# Get probabilities from the model
probabilities = loaded_model.predict(real_values)

# Extract probability for the "Diabetic" class (index 1)
diabetes_probability = probabilities[0][1]  

# Print result in Arabic
print(f"التوقع: {'Diabetic' if diabetes_probability >= 0.5 else 'Non-Diabetic'} ({diabetes_probability * 100:.2f}%)")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 78ms/step
التوقع: Non-Diabetic (0.00%)


In [4]:
import pandas as pd
import numpy as np

# تحديد عدد السجلات
num_samples = 1000

# توليد بيانات عشوائية ضمن نطاق القيم الحقيقية
np.random.seed(42)

data = {
    "HbA1c_level": np.round(np.random.uniform(4.5, 10.0, num_samples), 1),  # القيم بين 4.5 و 10
    "blood_glucose_level": np.random.randint(70, 250, num_samples),  # القيم بين 70 و 250
    "bmi": np.round(np.random.uniform(18.5, 40.0, num_samples), 1),  # القيم بين 18.5 و 40
    "age": np.random.randint(18, 90, num_samples),  # العمر بين 18 و 90 سنة
    "hypertension": np.random.choice([0, 1], num_samples, p=[0.85, 0.15]),  # 15% لديهم ارتفاع ضغط الدم
    "heart_disease": np.random.choice([0, 1], num_samples, p=[0.90, 0.10]),  # 10% لديهم أمراض القلب
}

# إنشاء DataFrame
df = pd.DataFrame(data)

# عرض أول 5 سجلات
print(df.head())


   HbA1c_level  blood_glucose_level   bmi  age  hypertension  heart_disease
0          6.6                  116  30.3   64             1              1
1          9.7                   81  39.3   75             1              0
2          8.5                  131  27.8   44             0              0
3          7.8                  149  25.2   70             0              0
4          5.4                  152  29.4   59             0              0


In [ ]:
import pandas as pd
import numpy as np

# تحديد عدد السجلات الأكبر لزيادة التوازن
num_samples = 5000  

# توليد بيانات عشوائية ضمن نطاق القيم الحقيقية
np.random.seed(42)

# توليد قيم HbA1c بتوزيع طبيعي حول القيم الشائعة
HbA1c_mean, HbA1c_std = 6.0, 1.5  # متوسط HbA1c حوالي 6 مع انحراف معياري 1.5
HbA1c_values = np.clip(np.random.normal(HbA1c_mean, HbA1c_std, num_samples), 4.5, 10.0)

# توليد قيم السكر في الدم بتوزيع طبيعي
glucose_mean, glucose_std = 120, 40  # متوسط السكر في الدم 120 مع انحراف معياري 40
glucose_values = np.clip(np.random.normal(glucose_mean, glucose_std, num_samples), 70, 250)

# توليد قيم مؤشر كتلة الجسم (BMI) بتوزيع طبيعي
bmi_mean, bmi_std = 27, 5  # متوسط BMI حوالي 27 مع انحراف 5
bmi_values = np.clip(np.random.normal(bmi_mean, bmi_std, num_samples), 18.5, 40.0)

# توليد قيم العمر بتوزيع طبيعي مع قص القيم غير الواقعية
age_mean, age_std = 50, 15  # متوسط العمر 50 مع انحراف 15
age_values = np.clip(np.random.normal(age_mean, age_std, num_samples), 18, 90)

# تحقيق التوازن في hypertension و heart_disease بحيث تكون النسب 50%-50%
hypertension_values = np.random.choice([0, 1], num_samples, p=[0.5, 0.5])
heart_disease_values = np.random.choice([0, 1], num_samples, p=[0.5, 0.5])

# إنشاء DataFrame
data = {
    "HbA1c_level": np.round(HbA1c_values, 1),
    "blood_glucose_level": np.round(glucose_values),
    "bmi": np.round(bmi_values, 1),
    "age": np.round(age_values),
    "hypertension": hypertension_values,
    "heart_disease": heart_disease_values
}

df = pd.DataFrame(data)

# عرض أول 5 سجلات
print(df.head())

print("\nHypertension Distribution:\n", df["hypertension"].value_counts(normalize=True))
print("\nHeart Disease Distribution:\n", df["heart_disease"].value_counts(normalize=True))


   HbA1c_level  blood_glucose_level   bmi   age  hypertension  heart_disease
0          6.7                103.0  23.6  48.0             0              0
1          5.8                102.0  25.5  50.0             1              1
2          7.0                 70.0  24.0  51.0             1              0
3          8.3                107.0  27.6  64.0             1              1
4          5.6                149.0  33.0  39.0             1              0

Hypertension Distribution:
 0    0.512
1    0.488
Name: hypertension, dtype: float64

Heart Disease Distribution:
 1    0.5034
0    0.4966
Name: heart_disease, dtype: float64


In [ ]:
import pandas as pd
import numpy as np

num_samples = 5000  

np.random.seed(42)

HbA1c_mean, HbA1c_std = 6.0, 1.5  
HbA1c_values = np.clip(np.random.normal(HbA1c_mean, HbA1c_std, num_samples), 4.5, 10.0)

glucose_mean, glucose_std = 120, 40  
glucose_values = np.clip(np.random.normal(glucose_mean, glucose_std, num_samples), 70, 250)

bmi_mean, bmi_std = 27, 5  
bmi_values = np.clip(np.random.normal(bmi_mean, bmi_std, num_samples), 18.5, 40.0)

age_mean, age_std = 50, 15  
age_values = np.clip(np.random.normal(age_mean, age_std, num_samples), 18, 90)

hypertension_values = np.random.choice([0, 1], num_samples, p=[0.5, 0.5])
heart_disease_values = np.random.choice([0, 1], num_samples, p=[0.5, 0.5])

diabetes_values = np.where(
    (HbA1c_values >= 6.5) | (glucose_values >= 126), 1, 0
)

data = {
    "HbA1c_level": np.round(HbA1c_values, 1),
    "blood_glucose_level": np.round(glucose_values),
    "bmi": np.round(bmi_values, 1),
    "age": np.round(age_values),
    "hypertension": hypertension_values,
    "heart_disease": heart_disease_values,
    "diabetes": diabetes_values
}

df = pd.DataFrame(data)

print(df.head())

print("\nDiabetes Distribution:\n", df["diabetes"].value_counts(normalize=True))


   HbA1c_level  blood_glucose_level   bmi   age  hypertension  heart_disease  \
0          6.7                103.0  23.6  48.0             0              0   
1          5.8                102.0  25.5  50.0             1              1   
2          7.0                 70.0  24.0  51.0             1              0   
3          8.3                107.0  27.6  64.0             1              1   
4          5.6                149.0  33.0  39.0             1              0   

   diabetes  
0         1  
1         0  
2         1  
3         1  
4         1  

Diabetes Distribution:
 1    0.6464
0    0.3536
Name: diabetes, dtype: float64


In [ ]:
import pandas as pd
import numpy as np

num_samples = 5000  

np.random.seed(42)

HbA1c_mean, HbA1c_std = 5.8, 1.3  
HbA1c_values = np.clip(np.random.normal(HbA1c_mean, HbA1c_std, num_samples), 4.5, 10.0)

glucose_mean, glucose_std = 115, 35  
glucose_values = np.clip(np.random.normal(glucose_mean, glucose_std, num_samples), 70, 250)

bmi_mean, bmi_std = 27, 5  
bmi_values = np.clip(np.random.normal(bmi_mean, bmi_std, num_samples), 18.5, 40.0)

age_mean, age_std = 50, 15  
age_values = np.clip(np.random.normal(age_mean, age_std, num_samples), 18, 90)

hypertension_values = np.random.choice([0, 1], num_samples, p=[0.5, 0.5])
heart_disease_values = np.random.choice([0, 1], num_samples, p=[0.5, 0.5])

diabetes_values = np.where(
    (HbA1c_values >= 6.5) | (glucose_values >= 126), 1, 0
)

unique, counts = np.unique(diabetes_values, return_counts=True)
diabetes_ratio = counts[1] / num_samples

if diabetes_ratio > 0.51:
    drop_indices = np.random.choice(df[df["diabetes"] == 1].index, size=int(counts[1] - 0.51 * num_samples), replace=False)
    df = df.drop(drop_indices)

elif diabetes_ratio < 0.49:
    drop_indices = np.random.choice(df[df["diabetes"] == 0].index, size=int(counts[0] - 0.49 * num_samples), replace=False)
    df = df.drop(drop_indices)

df = df.reset_index(drop=True)

data = {
    "HbA1c_level": np.round(HbA1c_values, 1),
    "blood_glucose_level": np.round(glucose_values),
    "bmi": np.round(bmi_values, 1),
    "age": np.round(age_values),
    "hypertension": hypertension_values,
    "heart_disease": heart_disease_values,
    "diabetes": diabetes_values
}

df = pd.DataFrame(data)

print(df.head())

print("\nDiabetes Distribution:\n", df["diabetes"].value_counts(normalize=True))


   HbA1c_level  blood_glucose_level   bmi   age  hypertension  heart_disease  \
0          6.4                100.0  23.6  48.0             0              0   
1          5.6                 99.0  25.5  50.0             1              1   
2          6.6                 70.0  24.0  51.0             1              0   
3          7.8                103.0  27.6  64.0             1              1   
4          5.5                141.0  33.0  39.0             1              0   

   diabetes  
0         0  
1         0  
2         1  
3         1  
4         1  

Diabetes Distribution:
 1    0.56
0    0.44
Name: diabetes, dtype: float64


In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

num_samples = 5000
np.random.seed(42)

HbA1c_values = np.clip(np.random.normal(5.8, 1.3, num_samples), 4.5, 10.0)
glucose_values = np.clip(np.random.normal(115, 35, num_samples), 70, 250)
bmi_values = np.clip(np.random.normal(27, 5, num_samples), 18.5, 40.0)
age_values = np.clip(np.random.normal(50, 15, num_samples), 18, 90)

diabetes_values = np.where((HbA1c_values >= 6.5) | (glucose_values >= 126), 1, 0)

df = pd.DataFrame({
    "HbA1c_level": np.round(HbA1c_values, 1),
    "blood_glucose_level": np.round(glucose_values),
    "bmi": np.round(bmi_values, 1),
    "age": np.round(age_values),
    "diabetes": diabetes_values
})

X = df.drop(columns=["diabetes"])
y = df["diabetes"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

model = keras.Sequential([
    keras.layers.Dense(16, activation="relu", input_shape=(X_train.shape[1],)),
    keras.layers.Dropout(0.3),  
    keras.layers.Dense(8, activation="relu"), 
    keras.layers.Dropout(0.3), 
    keras.layers.Dense(1, activation="sigmoid")  
])

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

history = model.fit(X_train_scaled, y_train, epochs=50, batch_size=32, validation_data=(X_test_scaled, y_test), verbose=1)

y_pred = (model.predict(X_test_scaled) > 0.5).astype("int32")

print("\nAccuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n", confusion_matrix(y_test, y_pred))


c:\Users\MINA\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\layers\core\dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.5991 - loss: 0.6550 - val_accuracy: 0.8850 - val_loss: 0.4495
Epoch 2/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.7806 - loss: 0.4816 - val_accuracy: 0.9110 - val_loss: 0.2918
Epoch 3/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8440 - loss: 0.3638 - val_accuracy: 0.9170 - val_loss: 0.2211
Epoch 4/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8571 - loss: 0.3227 - val_accuracy: 0.9240 - val_loss: 0.1869
Epoch 5/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8848 - loss: 0.2733 - val_accuracy: 0.9340 - val_loss: 0.1596
Epoch 6/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.8784 - loss: 0.2818 - val_accuracy: 0.9520 - val_loss: 0.1405
Epoch 7/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9070 - loss: 0.2228 - val_accuracy: 0.9610 - val_loss: 0.1204
Epoch 8/50
125/125 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step - accuracy: 0.9142 - loss: 0.2093 - val_accuracy: 0.

In [ ]:
real_life_samples = np.array([
    [7.2, 150, 30, 55],  
    [5.5, 90, 25, 40],   
    [8.0, 180, 35, 65],    
    [6.3, 110, 28, 50],   
])

real_life_samples_scaled = scaler.transform(real_life_samples)

predictions = model.predict(real_life_samples_scaled)

predicted_classes = (predictions > 0.5).astype("int32")

for i, (sample, pred) in enumerate(zip(real_life_samples, predicted_classes)):
    print(f"Sample {i+1}: HbA1c={sample[0]}, Glucose={sample[1]}, BMI={sample[2]}, Age={sample[3]} -> Diabetes Prediction: {'Positive' if pred[0] == 1 else 'Negative'}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step
Sample 1: HbA1c=7.2, Glucose=150.0, BMI=30.0, Age=55.0 -> Diabetes Prediction: Positive
Sample 2: HbA1c=5.5, Glucose=90.0, BMI=25.0, Age=40.0 -> Diabetes Prediction: Negative
Sample 3: HbA1c=8.0, Glucose=180.0, BMI=35.0, Age=65.0 -> Diabetes Prediction: Positive
Sample 4: HbA1c=6.3, Glucose=110.0, BMI=28.0, Age=50.0 -> Diabetes Prediction: Negative


c:\Users\MINA\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [19]:
# بيانات اختبار موسعة تحتوي على حالات متنوعة للسكري
real_life_samples = np.array([
    [7.5, 160, 32, 60],  # حالة متوسطة لمرض السكري
    [5.0, 85, 22, 35],   # شخص سليم
    [9.0, 200, 38, 70],  # حالة سكري حادة
    [6.1, 105, 26, 45],  # شخص في الحدود الطبيعية
    [10.0, 240, 40, 75], # مريض سكري بمؤشرات مرتفعة جدًا
    [4.8, 78, 19, 30],   # شخص سليم وصغير في العمر
    [7.0, 140, 29, 55],  # حالة قريبة من السكري
    [8.5, 190, 36, 65],  # مريض سكري مرتفع المخاطر
    [5.7, 95, 23, 42],   # شخص في الحد الطبيعي
    [6.8, 130, 27, 50],  # حالة غير مؤكدة (بين الطبيعي والسكري)
])

# تطبيع البيانات باستخدام StandardScaler
real_life_samples_scaled = scaler.transform(real_life_samples)

# توقع النتائج باستخدام النموذج المدرب
predictions = model.predict(real_life_samples_scaled)

# تحويل القيم لاحتمالات
predicted_classes = (predictions > 0.5).astype("int32")

# عرض النتائج
for i, (sample, pred) in enumerate(zip(real_life_samples, predicted_classes)):
    print(f"Sample {i+1}: HbA1c={sample[0]}, Glucose={sample[1]}, BMI={sample[2]}, Age={sample[3]} -> Diabetes Prediction: {'Positive' if pred[0] == 1 else 'Negative'}")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 24ms/step
Sample 1: HbA1c=7.5, Glucose=160.0, BMI=32.0, Age=60.0 -> Diabetes Prediction: Positive
Sample 2: HbA1c=5.0, Glucose=85.0, BMI=22.0, Age=35.0 -> Diabetes Prediction: Negative
Sample 3: HbA1c=9.0, Glucose=200.0, BMI=38.0, Age=70.0 -> Diabetes Prediction: Positive
Sample 4: HbA1c=6.1, Glucose=105.0, BMI=26.0, Age=45.0 -> Diabetes Prediction: Negative
Sample 5: HbA1c=10.0, Glucose=240.0, BMI=40.0, Age=75.0 -> Diabetes Prediction: Positive
Sample 6: HbA1c=4.8, Glucose=78.0, BMI=19.0, Age=30.0 -> Diabetes Prediction: Negative
Sample 7: HbA1c=7.0, Glucose=140.0, BMI=29.0, Age=55.0 -> Diabetes Prediction: Positive
Sample 8: HbA1c=8.5, Glucose=190.0, BMI=36.0, Age=65.0 -> Diabetes Prediction: Positive
Sample 9: HbA1c=5.7, Glucose=95.0, BMI=23.0, Age=42.0 -> Diabetes Prediction: Negative
Sample 10: HbA1c=6.8, Glucose=130.0, BMI=27.0, Age=50.0 -> Diabetes Prediction: Positive


c:\Users\MINA\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [20]:
# بيانات اختبار موسعة تحتوي على حالات متنوعة للسكري
real_life_samples = np.array([
    [7.5, 160, 32, 60],  # حالة متوسطة لمرض السكري
    [5.0, 85, 22, 35],   # شخص سليم
    [9.0, 200, 38, 70],  # حالة سكري حادة
    [6.1, 105, 26, 45],  # شخص في الحدود الطبيعية
    [10.0, 240, 40, 75], # مريض سكري بمؤشرات مرتفعة جدًا
    [4.8, 78, 19, 30],   # شخص سليم وصغير في العمر
    [7.0, 140, 29, 55],  # حالة قريبة من السكري
    [8.5, 190, 36, 65],  # مريض سكري مرتفع المخاطر
    [5.7, 95, 23, 42],   # شخص في الحد الطبيعي
    [6.8, 130, 27, 50],  # حالة غير مؤكدة (بين الطبيعي والسكري)
])

# تطبيع البيانات باستخدام StandardScaler
real_life_samples_scaled = scaler.transform(real_life_samples)

# توقع النتائج باستخدام النموذج المدرب
predictions = model.predict(real_life_samples_scaled)

# تحويل القيم لاحتمالات ونسبة مئوية
predicted_probabilities = predictions * 100  # تحويل القيم إلى نسبة مئوية
predicted_classes = (predictions > 0.5).astype("int32")  # تحويل إلى 0 أو 1

# عرض النتائج مع النسبة المئوية
for i, (sample, prob, pred) in enumerate(zip(real_life_samples, predicted_probabilities, predicted_classes)):
    print(f"Sample {i+1}: HbA1c={sample[0]}, Glucose={sample[1]}, BMI={sample[2]}, Age={sample[3]} -> "
          f"Diabetes Prediction: {'Positive' if pred[0] == 1 else 'Negative'} ({prob[0]:.2f}%)")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Sample 1: HbA1c=7.5, Glucose=160.0, BMI=32.0, Age=60.0 -> Diabetes Prediction: Positive (100.00%)
Sample 2: HbA1c=5.0, Glucose=85.0, BMI=22.0, Age=35.0 -> Diabetes Prediction: Negative (0.03%)
Sample 3: HbA1c=9.0, Glucose=200.0, BMI=38.0, Age=70.0 -> Diabetes Prediction: Positive (100.00%)
Sample 4: HbA1c=6.1, Glucose=105.0, BMI=26.0, Age=45.0 -> Diabetes Prediction: Negative (4.05%)
Sample 5: HbA1c=10.0, Glucose=240.0, BMI=40.0, Age=75.0 -> Diabetes Prediction: Positive (100.00%)
Sample 6: HbA1c=4.8, Glucose=78.0, BMI=19.0, Age=30.0 -> Diabetes Prediction: Negative (0.01%)
Sample 7: HbA1c=7.0, Glucose=140.0, BMI=29.0, Age=55.0 -> Diabetes Prediction: Positive (100.00%)
Sample 8: HbA1c=8.5, Glucose=190.0, BMI=36.0, Age=65.0 -> Diabetes Prediction: Positive (100.00%)
Sample 9: HbA1c=5.7, Glucose=95.0, BMI=23.0, Age=42.0 -> Diabetes Prediction: Negative (0.64%)
Sample 10: HbA1c=6.8, Glucose=130.0, BMI=27.0, Age=50.0 -> Diabetes Prediction: Positive (

c:\Users\MINA\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


In [11]:
model.save("C:/Users/MINA/Downloads/Createdmodel.h5")